# Advanced Python
In this lecture we look at a lot of language features that make Python special compared to other programming languages. 

# The Python Data Model
Under the hood, most Python syntax is just *syntatic sugar* for method calls. If you know the methods that are implicitly called by the common syntax you can design objects that beautifully integrate with the language.

In [1]:
3 + 3 

6

is in fact doing 

In [2]:
(3).__add__(3)

6

Let's find out how we can make objects that behave nicely with the rest of the language. As an example we look at a class for representing triples of numeric values.

In [3]:
class Triple:
    def __init__(self, num1, num2, num3):
        self.nums = num1, num2, num3

In [4]:
Triple(1, 2, 3)

The string representation of our class is not really informative. We can fix this by implementing `__repr__`.

In [5]:
class Triple:
    def __init__(self, num1, num2, num3):
        self.nums = num1, num2, num3
        
    def __repr__(self):
        """A string representation for inspecting objects at runtime."""
        return "Triple(" + str(self.nums[0]) + ", " + str(self.nums[1]) + ", " + str(self.nums[2]) + ")"
    
Triple(1, 2, 3)

Triple(1, 2, 3)

To make addition between `Triple`s possible, we have to implement `__add__`. We define the addition `Triple`s just as the elementwise addition of the three numbers.  

In [6]:
class Triple:
    def __init__(self, num1, num2, num3):
        self.nums = num1, num2, num3
        
    def __repr__(self):
        """A string representation for inspecting objects at runtime."""
        return "Triple(" + str(self.nums[0]) + ", " + str(self.nums[1]) + ", " + str(self.nums[2]) + ")"
    
    def __add__(self, other):
        num1 = self.nums[0] + other.nums[0]
        num2 = self.nums[1] + other.nums[1]
        num3 = self.nums[2] + other.nums[2]
        return Triple(num1, num2, num3)
        
    
a = Triple(1, 2, 3)
b = Triple(2, 3, 4)

Because we implemented `__add__` we can add triples with the plus operator.
The following three expressions are all the same! The first one is the fast way to write it, which 
internally maps to the second, which internally maps to the third!


In [7]:
print(a + b)
print(a.__add__(b))
print(Triple.__add__(a, b))

Triple(3, 5, 7)
Triple(3, 5, 7)
Triple(3, 5, 7)


Now it would be nice to enable addition between triples and scalar numbers by just elementwise adding the scalar to all three triple values. But the expression

In [8]:
1 + Triple(1, 2, 3)

TypeError: unsupported operand type(s) for +: 'int' and 'Triple'

is interpreted as 

In [9]:
int.__add__(1, Triple(1, 2, 3))

NotImplemented

which return the special value `NotImplemented`. If a binary operaton does not work when called on the first operand does not work, Python tries to invert the order of operands, calling `__radd__` on the other. If this does not work either a `TypeError` is raised. By implementing `__radd__` we can make scalar addition work without changing the behaivor of the `int`s.

In [10]:
class Triple:
    def __init__(self, num1, num2, num3):
        self.nums = num1, num2, num3
        
    def __repr__(self):
        """A string representation for inspecting objects at runtime."""
        return "Triple(" + str(self.nums[0]) + ", " + str(self.nums[1]) + ", " + str(self.nums[2]) + ")"
    
    def __add__(self, other):
        num1 = self.nums[0] + other.nums[0]
        num2 = self.nums[1] + other.nums[1]
        num3 = self.nums[2] + other.nums[2]
        return Triple(num1, num2, num3)
    
    def __radd__(self, other):
        num1 = self.nums[0] + other
        num2 = self.nums[1] + other
        num3 = self.nums[2] + other
        return Triple(num1, num2, num3)

In [11]:
1 + Triple(1, 2, 3)

Triple(2, 3, 4)

## Exercise
Make the `in` operator work on our triples. For that we need to implement `__contains__`. The statement 
```python 
3 in Triple(1, 2, 3)
```

should become `True` after defining `__contains__`.

In [12]:
class Triple:
    def __init__(self, num1, num2, num3):
        self.nums = num1, num2, num3
        
    def __repr__(self):
        """A string representation for inspecting objects at runtime."""
        return "Triple(" + str(self.nums[0]) + ", " + str(self.nums[1]) + ", " + str(self.nums[2]) + ")"
    
    def __contains__(self, item):
        return item in self.nums

In [13]:
3 in Triple(1, 2, 3)

True

You can read more about Python's data model at https://docs.python.org/3/reference/datamodel.html.

# Iterables and Iterators
Objects that can be used in `for ... in ...` statements are called *iterable*.

In [14]:
class Triple:
    def __init__(self, num1, num2, num3):
        self.nums = num1, num2, num3
    
    def __repr__(self):
        """A string representation for inspecting objects at runtime."""
        return "Triple(" + str(self.nums[0]) + ", " + str(self.nums[1]) + ", " + str(self.nums[2]) + ")"
    
    def __iter__(self):
        return iter(self.nums)
    
my_triple = Triple(1, 2, 3)


for value in my_triple:
    print(value)


1
2
3


The `__iter__` - magic-method is what makes an object iterable. Behind the scenes, the `iter`-function calls this methods to get the iterator.

An *iterator* is an object that implements `__next__`. Usually, you want to make an iterator also iterable by returning itself from `__iter__`. Here an example of how to create your own `range`-function:

In [15]:
class myrange:
    def __init__(self, n):
        self.i = 0
        self.n = n

    def __iter__(self):
        return self

    def __next__(self):
        if self.i < self.n:
            self.i += 1
            return self.i
        else:
            raise StopIteration()

In [16]:
for i in myrange(5):
    print(i)

1
2
3
4
5


Python relies heavily on iterators, and you should use them everytime Python offers them! the following code would be considered *unpythonic*.

In [17]:
a_list = [10, 20, 30]
for i in range(len(a_list)):
    print(a_list[i])

10
20
30


Instead we prefer using the iterator directly.

In [18]:
a_list = [10, 20, 30]
for number in a_list:
    print(number)

10
20
30


# Properties
Other languages often define *getters* and *setters* to restrict access to object attributes. In Python we can add getter and setter logic using `properties`.

In [19]:
class Triple():
    def __init__(self, num1, num2, num3):
        self._nums = num1, num2, num3
      
    @property
    def nums(self):
        return self._nums

    
a = Triple(1, 2, 3)
a.nums

(1, 2, 3)

By default, we can not assign to attributes that are declared via properties. 

In [20]:
a.nums = 10, 11, 12

AttributeError: can't set attribute

But we can add a setter with another decorator. This is useful for including validation logic.

In [21]:
class Triple():
    def __init__(self, num1, num2, num3):
        self._nums = num1, num2, num3
      
    @property
    def nums(self):
        return self._nums
    
    @nums.setter
    def nums(self, value):
        if len(value) == 3:
            self._nums = value
        else:
            raise ValueError("Three values are required to set the data.")
    
a = Triple(1, 2, 3)
a.nums = (4, 5, 6)
a.nums

(4, 5, 6)

With `properties` we can add getter and setter logic, without having them surface in our objects interfaces. This also means you can start to write 
your classes with plain attributes and add getters and setters later if required.

# Factory methods
Sometimes we want to be able to initialize our objects in different ways. A classic pattern of object oriented programming
is the `Factory`. A `Factory`'s only purpose is to initialize other objects. In Python, we do not really need this
pattern, but instead we can use *factory methods* to initialize our object in different ways. For that we can use
`classmethods` that take the class instead of the instance as a first argument.

In [22]:
class Triple():
    def __init__(self, num1, num2, num3):
        self._nums = num1, num2, num3
    
    def __repr__(self):
        """A string representation for inspecting objects at runtime."""
        return "Triple(" + str(self.nums[0]) + ", " + str(self.nums[1]) + ", " + str(self.nums[2]) + ")"
    
    @property
    def nums(self):
        return self._nums
    
    @classmethod
    def from_value(cls, num):
        return cls(num, num, num)

Triple.from_value(3)

Triple(3, 3, 3)

# Generators

A Python generator function is a function which returns a generator. Generator functions a implicitly defined by the use of `yield` in the function body. `yield` may be used with a value, in which case that value is treated as the "generated" value. The next time `next()` is called on the generator (i.e. in the next step in a for loop, for example), the generator resumes execution from where it called `yield`, not from the beginning of the function. All of the state, like the values of local variables, is recovered and the generator contiues to execute until the next call to `yield`. 

https://jeffknupp.com/blog/2013/04/07/improve-your-python-yield-and-generators-explained/

In [23]:
def generate_numbers():
    yield 1
    yield 10
    yield 3
    yield 5
    
for i in generate_numbers():
    print(i)

1
10
3
5


In [24]:
a = generate_numbers()
print(a)

print(next(a))
print()

for i in a:
    print(i)
    
print(next(a)) #will throw a StopIteration

<generator object generate_numbers at 0x7eff50c14048>
1

10
3
5


StopIteration: 

When we call a normal Python function, execution starts at function's first line and continues until a return statement, exception, or the end of the function is encountered. 
Once a function returns control to its caller, any work done by the function and stored in local variables is lost. A new call to the function creates everything from scratch. 

A **generator** is a certain kind of function (recognized by the keyword *yield* in place of *return*), that doesn't lose its data. If a generator is called, it will run until the next occurence of the `yield` keyword. When called again, it starts right after that, and runs until the next occurence of `yield`.

A generator is an iterator, which means you can loop over it, call next(), and use it the way you'd use any other iterator

In [25]:
hasattr(a, '__iter__'), hasattr(a, '__next__')

(True, True)

## Exercise
Use a generator ot produce even number infinitely. Then print the first ten even numbers.

Use a `while True` loop to produces number infinitely. Then wrap the generator function in `enumerate` and `break` after the first ten values. 

In [26]:
def even_numbers():
    i = 0
    while True:
        if i % 2 == 0:
            yield i
        
        i += 1

for i, num in enumerate(even_numbers()):
    print(num)
    if i >= 10:
        break

0
2
4
6
8
10
12
14
16
18
20


# Context managers (and IO)
Context managers can be seen as conceptual counterpart to functions. While a function presents a chunk of code that is reused in between other operations, a context manager is a chunk of code that is reused *around* other operations.

In [27]:
class PrintingContext:
    
    def __enter__(self):
        print('Entering context.')
    
    def __exit__(self, exception_type, exception_value, traceback):
        print('Exiting context.')
        
        
with PrintingContext():
    print('I am inside the context')

Entering context.
I am inside the context
Exiting context.


## File IO

Writing to a file.

In [28]:
string = """hello world!
this is chris, and I am writing 
this message!
"""
fh = open('test.txt', 'w') # open needs as arguments the file-path, and a mode ("r": read, "w": write, "a": append, 
                           #                                                    "rb": read binary, "wb": write binary, "a": append binary) 
                           # and returns a file-handle we can work with
fh.write(string)
fh.close()                 # don't forget to close the file afterwards!

In [29]:
%%bash
cat test.txt

hello world!
this is chris, and I am writing 
this message!


Now we can read from the file what we just wrote in there.

In [30]:
# reading example:
fh = open('test.txt', 'r')
lines = fh.readlines()

for line in lines:
    print(line, end='')

fh.close()

hello world!
this is chris, and I am writing 
this message!


## Better done with context managers
Context managers are really useful for handling resources that need to released after they are no longer used. The prototypical example is file IO.

In [31]:
with open('save_file.txt', mode='w') as file_context:    # __enter__ is called here.
    file_context.write('You cannot forget to close me.')
# __exit__ is called here.

In [32]:
class File():

    def __init__(self, filename, mode):
        self.filename = filename
        self.mode = mode

    def __enter__(self):
        self.open_file = open(self.filename, self.mode)
        return self.open_file

    def __exit__(self, *args):
        self.open_file.close()

with File('save_file.txt', mode='r') as fh:
    print(fh.readlines())

['You cannot forget to close me.']


more info on context managers: https://jeffknupp.com/blog/2016/03/07/python-with-context-managers/

# Exceptions

In [33]:
import random

In [34]:
a = [1, 2, 3] if random.randint(0,1) else 1

first_val = a[0] #throws an Exception in 50% of cases

In [35]:
a = [1, 2, 3] if random.randint(0,1) else 1

# we can catch that exception! In Java, this is try-catch, in python it's called try-except
try:
    first_val = a[0]
    print("everything worked!")
except Exception as e:
    print(type(e), e)

<class 'TypeError'> 'int' object is not subscriptable


In [36]:
try:
    file_handle = open('test.txt')
except FileNotFoundError as err:
    print('this will be executed if a FileNotFoundError occurs')
    print(err)
finally:
    print('this will be executed whether the try block throws an error or not')
    file_handle.close()


this will be executed whether the try block throws an error or not


In [37]:
# Exceptions will go up through functions if unhandled
def foo():
    try:
        [1, 2][3] #this will cause an IndexError, however as it isn't handled here, the error is thrown upward to the caller
        open('asdf')
    except FileNotFoundError as err:
        print('file not found error')

try:
    foo()
    print("won't be reached")
except IndexError as err:
    print('index error')

index error


In [38]:
# you can catch multiple exceptions in one try-except statement

try:
    [1,2][3]
except Exception:         #it will start chronologically at the first one, looking if this fits....
    print("this will run") 
except IndexError:        #and if it does, it won't execute the others
    print("this won't..")


this will run


![exceptions](./errors.png)

`try-except` also has an 'else', which runs if no error was thrown.

In [39]:
try:
    randval = random.randint(0,2)
    print("randval is:", randval)
    if randval == 0:
        [1,2][3]
    elif randval == 1:
        5/0
except IndexError: 
    print("this will run if randval was 0") 
except ZeroDivisionError:       
    print("this will run if randval was 1")
else:
    print("this will run if randval was 2")

randval is: 2
this will run if randval was 2


In [40]:
# You can even extend Exception yourself, to throw your own errors!

class NotTheValueIWantedException(Exception):
    pass

print(isinstance(NotTheValueIWantedException(), Exception))

True


In [41]:
def my_method(value):
    if value != 42 and value != 1337:
        raise NotTheValueIWantedException
        
for i in range(2000):
    try:
        my_method(i)
        print("A value it accepted was:", i)
    except NotTheValueIWantedException:
        pass

A value it accepted was: 42
A value it accepted was: 1337


# Decorators
Decorators a functions that change the functionality of other functions or classes. This should be usually be done in a transparent manner, i.e. the interface of the original function stays the same, while the functionaliy is added around it.

In [42]:
def substract(x, y):
    return x - y

def decorated_substract(*args, **kwargs):
    result = substract(*args, **kwargs)              
    print('~~~ result of', substract.__name__, '~~~')
    print(result)                               
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~')        
    return result    
    
decorated_substract(5, 2)

~~~ result of substract ~~~
3
~~~~~~~~~~~~~~~~~~~~~~~~~~~


3

This however only creates a new function, containing a changed behaviour of the substract-function. What if we want to change the behaviour of arbitrary functions?

In [43]:
def substract(x, y):
    return x - y

def add(x, y):
    return x + y

def decorated(func, *args, **kwargs):
    result = func(*args, **kwargs)              
    print('~~~ result of', func.__name__, '~~~')
    print(result)                               
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~')        
    return result    
    
decorated(add, 5, 2)

~~~ result of add ~~~
7
~~~~~~~~~~~~~~~~~~~~~~~~~~~


7

We're still not okay with this, because we want to change the behaviour of `add` itself!

In [44]:
def print_decorator(func):                           # func is the method which will be decorated by this
        
    print("This occurs when we re-define the function")
    
    #if we define function = decorated(function), the new function will be this:
    
    def inner(*args, **kwargs):                      # we define a new function here, taking any parameters...
        result = func(*args, **kwargs)               # which, when called, executes the original function with these parameters...
        print('~~~ result of', func.__name__, '~~~') # prints name of original funciton...
        print(result)                                # prints the result of the function...
        print('~~~~~~~~~~~~~~~~~~~~~~~~~~~')         # some lines...
        return result                                # and returns that result of that function 
    
    return inner   # the new function is this inner function!

In [45]:
decorated_add = print_decorator(add)

This occurs when we re-define the function


In [46]:
decorated_add(3, 5)

~~~ result of add ~~~
8
~~~~~~~~~~~~~~~~~~~~~~~~~~~


8

In [47]:
add = print_decorator(add)
add(3,5)

This occurs when we re-define the function
~~~ result of add ~~~
8
~~~~~~~~~~~~~~~~~~~~~~~~~~~


8

Python provides a syntax for the assignment `function = decorated(function)`. This however just *syntactic sugar* for calling the decorator directly. 

In [48]:
@print_decorator
def multiply(x, y):
    return x * y

multiply(3, 5)
multiply(4, 5)

This occurs when we re-define the function
~~~ result of multiply ~~~
15
~~~~~~~~~~~~~~~~~~~~~~~~~~~
~~~ result of multiply ~~~
20
~~~~~~~~~~~~~~~~~~~~~~~~~~~


20

We can even chain decorators!

In [49]:
def bold(fn):
    """wraps the result of a function such that it's bold"""
    def wrapped():
        return "<b>" + fn() + "</b>"
    return wrapped

def italic(fn):
    """wraps the result of a function such that it's italics"""
    def wrapped():
        return "<i>" + fn() + "</i>"
    return wrapped

@bold
@italic
def hello():
    """prints 'hello world'"""
    return "hello world"

hello()

'<b><i>hello world</i></b>'

In [50]:
hello?

That's it almost it for basic knowledge of decorators!
There's just one important thing: If we replace the original function by the decorated version, we lose all information of the orginal function, as its docstring, information about arguments, etc. To make up for that, we use *another decorator*, namely `functools.wraps`. This simply copies the docstring of the original function to the new one.

In [51]:
from functools import wraps
from IPython.display import HTML

def html(fn):
    @wraps(fn)
    def wrapped():
        return HTML(fn())
    return wrapped


def bold(fn):
    @wraps(fn)
    def wrapped():
        return "<b>" + fn() + "</b>"
    return wrapped

def italic(fn):
    @wraps(fn)
    def wrapped():
        return "<i>" + fn() + "</i>"
    return wrapped

@html
@bold
@italic
def hello():
    """prints 'hello world'"""
    return "hello world"

hello()

In [52]:
hello?

## Exercise
Define and apply a decorator that makes a string appear red. You can achieve this by wrapping the string in `<span style='color: red'> str </span>`

In [53]:
def red(fn):
    @wraps(fn)
    def wrapped():
        return "<span style='color: red'>" + fn() + "</span>"
    return wrapped

@html
@red
@makebold
@makeitalic
def hello():
    """prints 'hello world'"""
    return "hello world"

hello()

NameError: name 'makebold' is not defined

# Lambda Expressions

Lambda expressions can be used to create "small", "throw-away", anonymous functions.

In [54]:
def square_number(x):
    return x**2

square_number(8), type(square_number)

(64, function)

In [55]:
square_number = lambda x: x**2

square_number(8), type(square_number)

(64, function)

In [56]:
calc_sum = lambda x, y: x + y
calc_sum(2, 3)

5

## Controlling list operations with lambdas

In [57]:
unsorted_list = [6, 1, 45, 67, 3, 7]

# two ways to sort:
new_list = sorted(unsorted_list) # creates a new sorted one, old one stays the same
unsorted_list.sort()             # sorts in-place, the old one will change

print(new_list)
print(unsorted_list)

[1, 3, 6, 7, 45, 67]
[1, 3, 6, 7, 45, 67]


Sorting in  descending order.

In [58]:
unsorted_list.sort(reverse=True) 
unsorted_list

[67, 45, 7, 6, 3, 1]

Sorting according to specific rules can be done with lambda functions. For example you can sort people by their age.

In [59]:
people = [
    {'name': 'Aaron', 'age': 40},
    {'name': 'Berta', 'age': 20},
    {'name': 'Chris', 'age': 29},
]

In [60]:
people.sort(key=lambda item: item['age'])
people

[{'name': 'Berta', 'age': 20},
 {'name': 'Chris', 'age': 29},
 {'name': 'Aaron', 'age': 40}]

or by their name.

In [61]:
people.sort(key=lambda item: item['name'])
people

[{'name': 'Aaron', 'age': 40},
 {'name': 'Berta', 'age': 20},
 {'name': 'Chris', 'age': 29}]

Other functions work similarly, For example you can use the `key` argument to turn `max` into `argmax`.

In [62]:
max(people, key=lambda x: x['age'])

{'name': 'Aaron', 'age': 40}

## Exercise
Use the `min` function with a `key` argument to find the person that comes first in the alphabet.

In [63]:
min(people, key=lambda x: x['name'])

{'name': 'Aaron', 'age': 40}

# Comprehensions
Comprehensions are a special syntax that simplify the creation of collections.

## List comprehension

To get a list of of squares from a list of numbers we would usually have to write code like this.

In [64]:
original_numbers = [1, 2, 3, 4, 5]
squared_numbers = []
for i in original_numbers:
    squared_numbers.append(i**2)
    
squared_numbers

[1, 4, 9, 16, 25]

However, with a comprehension, we can greatly simplify this.

In [65]:
squared_numbers = [i**2 for i in original_numbers]
squared_numbers

[1, 4, 9, 16, 25]

The general syntax for a comprehension is `[`expression `for` element `in` iterable \[`if` filter_condition\] `]`. Let's see this in action.

In [66]:
original_values = [(1, True), (2, False), (3, False), (4, True), (5, False), (7, True)]
only_trues = []
for i in original_values:
    if i[1]:
        only_trues.append(i[0])

only_trues

[1, 4, 7]

In [67]:
only_trues = [
    i[0]                      # what to do with the values from the old list
    for i in original_values  # for-loop like syntax
    if i[1]                   # filtering. 
]
only_trues

[1, 4, 7]

If we do not just want to filter, but instead do something else with the values that do not satisfy our filter condition, we can use a 
ternary expression.

In [68]:
only_trues_or_zero = [
    i[0] if i[1] else 0       # what to do with the values from the old list
    for i in original_values  # for-loop like syntax
]
only_trues_or_zero

[1, 0, 0, 4, 0, 7]

## Dictionary Comprehension

In [69]:
numbers_and_their_squares = {num: num ** 2 for num in [1,2,3,4,5]}
numbers_and_their_squares

{1: 1, 2: 4, 3: 9, 4: 16, 5: 25}

## Generator Comprehension
Generator comprehension is a compact way to write down generators

In [70]:
a = (i for i in range(10))
print(a) # it's a generator!
next(a)
print(next(a))
print(list(a))

<generator object <genexpr> at 0x7eff50c14a98>
1
[2, 3, 4, 5, 6, 7, 8, 9]


# Strings and format-strings

## some string-operations

In [71]:
# remove whitespaces at beginning and end
s = "   hello    "
new_s = s.strip()

# split string in multiple parts (gives a list)
s = "test, hello, world"
splitted = s.split(', ')
print(splitted, type(splitted))

# create a string again by joining a list of partial strings
', '.join(splitted) # -> "test, hello, world"

['test', 'hello', 'world'] <class 'list'>


'test, hello, world'

## Format-strings: version 1

Standard format-strings are C-style. The `%`-operator formats a set of varaibles in a tuple together with a special format-strings. This syntax is still from Python 2 and there is no good reason to still use it today.

In [72]:
import math

an_int = 15
a_float = math.pi #3.141592653589793
a_string = "string!"

string = "An int: %d, a rounded float: %.2f, a string: %s" % (an_int, a_float, a_string)
print(string)

An int: 15, a rounded float: 3.14, a string: string!


## Format-strings: version 2

Python's native format()-method is far more powerful than the borrowed c-syntax. For a complete list of it's arguments, look at https://docs.python.org/3.4/library/string.html#formatspec. For a nice small overview, have a look at https://wiki.python.org/moin/FormatReference

In [73]:
people = [
    {'name': 'aaron', 'age': 40},
    {'name': 'berta', 'age': 20},
    {'name': 'chris', 'age': 21},
]

# the {} are placeholders and are filled with the arguments of format()
for person in people:
    print("{} is {} years old!".format(person['name'], person['age']))

aaron is 40 years old!
berta is 20 years old!
chris is 21 years old!


The `{}` placeholders can be given names to make order irrelevant.

In [74]:
for person in people:
    print("{name} is {age} years old!".format(
        age=person['age'],
        name=person['name']))

aaron is 40 years old!
berta is 20 years old!
chris is 21 years old!


In [75]:
# names allow for neat dict-unpacking into the format() function

for person in people:
    print("{name} is {age} years old!".format(**person))

aaron is 40 years old!
berta is 20 years old!
chris is 21 years old!


In [76]:
# format() allows for many many formatting options, such as justification and
# conversion to decimal places etc...
nums = [10**i for i in range(5)] # -> [1, 10, 100, 1000, 10000]

for n in nums:
    # >     right justified
    # 10    10 characters long
    # .2    show 2 decimal places
    # f     display as floating point number
    print("{:>10.2f}".format(n))

      1.00
     10.00
    100.00
   1000.00
  10000.00


## Format-strings: version 3

Since Python 3.6, there is nice super-easy way to produce formatted string literals: `fstrings`! `fstrings` are very readable and should be the preferred way to format strings, except if you need to do something very complex that requires `str.format`.

In [77]:
name = "Fred"
string = f"He said his name is {name}."
string

'He said his name is Fred.'

In [78]:
f"12 + 16 = {12 + 16}"

'12 + 16 = 28'

In [79]:
width = 10
precision = 4
value = 12.34567
f"result: {value:{width}.{precision}}" 

'result:      12.35'